In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from scipy import optimize
from scipy import stats
import scipy
from datetime import datetime
from time import time
from scipy.stats import iqr
from sklearn.metrics import roc_auc_score
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import rankdata

# Synthetization of 3,600 data cubes

In [ ]:
for n in range (1,10):
    exec(f'd{n} = pd.read_csv(r"C:\\__datacube{n}.csv")')
    exec(f'd{n}["Outlier"]=0')
    
#8*3*30*5 = 3,600 data cubes   
for n in range (2,10):
    for p in [25, 100, 500]:  
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                random.seed(r)
                #Splitting data in Noise and Outlier
                exec(f'd{n}_o_{p}p_{r}rs = d{n}.sample(frac=({p}/10000), replace=False, random_state={r})')
                exec(f'd{n}_n_{p}p_{r}rs = d{n}.drop(d{n}_o_{p}p_{r}rs.index)')
                exec(f'd{n}_o_{p}p_{r}rs["Outlier"]=1')

                #Calculating IQR and mean on the non-outliers
                for dim in ["Time", "City", "Product"]:
                    exec(f'data_IQR = d{n}_n_{p}p_{r}rs[[dim, "Sales"]]')
                    IQR = data_IQR.groupby(dim).apply(iqr)
                    IQR = IQR.reset_index()
                    IQR = IQR.rename(columns={0:"IQR_"+dim})
                    IQR["IQR_"+dim] = IQR["IQR_"+dim].astype(float)
                    exec(f'd{n}_o_{p}p_{r}rs = pd.merge(d{n}_o_{p}p_{r}rs, IQR, on = [dim], how = "left")')

                for dim in ["Time", "City", "Product"]:
                    exec(f'data_mean = d{n}_n_{p}p_{r}rs[[dim, "Sales"]]')
                    Mean = data_mean.groupby(dim).apply(np.mean)
                    Mean = Mean.reset_index()
                    Mean = Mean.rename(columns={"Sales":"Mean_"+dim})
                    exec(f'd{n}_o_{p}p_{r}rs = pd.merge(d{n}_o_{p}p_{r}rs, Mean, on = [dim], how = "left")')
                
                #Outlier = value nearest to genreated random value, that is an outlier in one dimension 
                exec(f'd{n}_o_{p}p_{r}rs["O_Time_p"]= np.ceil(d{n}_o_{p}p_{r}rs["Mean_Time"] + (2*d{n}_o_{p}p_{r}rs["IQR_Time"]))')
                exec(f'd{n}_o_{p}p_{r}rs["O_Time_n"]= np.floor(d{n}_o_{p}p_{r}rs["Mean_Time"] - (2*d{n}_o_{p}p_{r}rs["IQR_Time"]))')
                exec(f'd{n}_o_{p}p_{r}rs["O_City_p"]= np.ceil(d{n}_o_{p}p_{r}rs["Mean_City"] + (2*d{n}_o_{p}p_{r}rs["IQR_City"]))')
                exec(f'd{n}_o_{p}p_{r}rs["O_City_n"]= np.floor(d{n}_o_{p}p_{r}rs["Mean_City"] - (2*d{n}_o_{p}p_{r}rs["IQR_City"]))')
                exec(f'd{n}_o_{p}p_{r}rs["O_Product_p"]= np.ceil(d{n}_o_{p}p_{r}rs["Mean_Product"] + (2*d{n}_o_{p}p_{r}rs["IQR_Product"]))')
                exec(f'd{n}_o_{p}p_{r}rs["O_Product_n"]= np.floor(d{n}_o_{p}p_{r}rs["Mean_Product"] - (2*d{n}_o_{p}p_{r}rs["IQR_Product"]))')
                
                
                exec(f'd{n}_o_{p}p_{r}rs["Diff_Time_p"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_Time_p"])')
                exec(f'd{n}_o_{p}p_{r}rs["Diff_Time_n"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_Time_n"])')
                exec(f'd{n}_o_{p}p_{r}rs["Diff_City_p"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_City_p"])')
                exec(f'd{n}_o_{p}p_{r}rs["Diff_City_n"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_City_n"])')
                exec(f'd{n}_o_{p}p_{r}rs["Diff_Product_p"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_Product_p"])')
                exec(f'd{n}_o_{p}p_{r}rs["Diff_Product_n"]= np.absolute(d{n}_o_{p}p_{r}rs["Sales"]-d{n}_o_{p}p_{r}rs["O_Product_n"])')
                
                def get_col(sr):
                    name=np.abs(sr).idxmin()
                    return pd.Series([name])
                exec(f'd{n}_o_{p}p_{r}rs[["Min_col"]] = d{n}_o_{p}p_{r}rs[["Diff_Time_p", "Diff_Time_n", "Diff_City_p", "Diff_City_n", "Diff_Product_p", "Diff_Product_n"]].apply(lambda x : get_col(x), axis=1)')
                exec(f'd{n}_o_{p}p_{r}rs.Min_col.replace(["Diff_Time_p", "Diff_Time_n", "Diff_City_p", "Diff_City_n", "Diff_Product_p", "Diff_Product_n"], ["O_Time_p", "O_Time_n", "O_City_p", "O_City_n", "O_Product_p", "O_Product_n"], inplace=True)')
                
                exec(f'len_df = len(d{n}_o_{p}p_{r}rs)')
                for i in range(len_df):
                    exec(f'd{n}_o_{p}p_{r}rs.loc[i, "Sales"] = d{n}_o_{p}p_{r}rs.loc[i, d{n}_o_{p}p_{r}rs.loc[i, "Min_col"]]')
                
                #Drop columns that were necessary for new Sales calculation
                exec(f'd{n}_o_{p}p_{r}rs = d{n}_o_{p}p_{r}rs.iloc[:, :-19]')
                
                #Adding Noise and Outlier dataframe
                exec(f'd{n}_{p}p_{r}rs = pd.concat([d{n}_o_{p}p_{r}rs, d{n}_n_{p}p_{r}rs])')
                #exec(f'd{n}_{p}p_{r}rs.sort_index(inplace = True)')
                
                #Adding Noise to overall dataframe
                exec(f'd{n}_{p}p_{r}rs_std = np.std(d{n}_{p}p_{r}rs["Sales"])')
                exec(f'd{n}_{p}p_{r}rs["Sales"] = d{n}_{p}p_{r}rs["Sales"] + (d{n}_{p}p_{r}rs_std/3*np.round(np.random.normal(0,{s}),0))')
                
                #Eliminating negative values
                exec(f'd{n}_{p}p_{r}rs.loc[d{n}_{p}p_{r}rs["Sales"] <0, "Sales"] =1')

# Key figures of the data cubes

In [3]:
df = []
for n in range (2,10):
    for p in [25, 100, 500]:  
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basisdataset = n
                Percentage_Outliers = p/100
                exec(f'Mean = d{n}_{p}p_{r}rs["Sales"].mean()')
                exec(f'Std = np.std(d{n}_{p}p_{r}rs["Sales"])')
                exec(f'Min = d{n}_{p}p_{r}rs["Sales"].min()')
                exec(f'Max = d{n}_{p}p_{r}rs["Sales"].max()')
                exec(f'Twentyfive = np.quantile(d{n}_{p}p_{r}rs["Sales"], 0.25)')
                exec(f'Seventyfive = np.quantile(d{n}_{p}p_{r}rs["Sales"], 0.75)')
                exec(f'Length = len(d{n}_{p}p_{r}rs)')
                df.append((Dataset, Basisdataset, Percentage_Outliers, Mean, Std, Min, Max, Twentyfive, 
                           Seventyfive, Length))
                cols=["Dataset", "Basisdataset", "Percentage_Outliers", "Mean", "Std", "Min", "Max", "Twentyfive", 
                           "Seventyfive", "Length"]
                summary = pd.DataFrame(df, columns=cols)
                
#summary.to_excel(r"C:\\__Df_summary.xlsx")

In [4]:
datasets = summary.groupby("Basisdataset").mean()
datasets = datasets.reset_index()
display(datasets)
#datasets.to_excel(r"C:\\__Basisdatasets.xlsx")

,Basisdataset,Percentage_Outliers,Mean,Std,Min,Max,Twentyfive,Seventyfive,Length
0,2,2.083333,304.859730,31.783748,225.875246,397.489639,282.602238,326.268429,3000.0
1,3,2.083333,176.192907,14.084944,138.281968,217.921682,165.563517,185.378686,6000.0
2,4,2.083333,245.463408,25.237258,180.520880,321.051279,227.921770,262.436588,6000.0
3,5,2.083333,245.792338,25.324844,180.241343,326.296899,228.249790,262.955669,12000.0
4,6,2.083333,454.693433,42.001195,344.118347,598.374781,425.089439,482.508792,13000.0
5,7,2.083333,99.169131,7.544260,78.015224,122.581890,93.917457,104.410439,26000.0
6,8,2.083333,259.605273,19.532013,205.214834,318.814834,245.833961,273.014342,26000.0
7,9,2.083333,78.029411,7.660797,55.984554,102.796726,72.367382,83.189604,52000.0


# RODD S-75

In [5]:
start = time()
ds = []
for n in range (2,10):
    for p in [25,100,500]: 
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                num_v = "Sales"
                cat_dims = ["Time", "City", "Product"]
                exec(f'data_log = d{n}_{p}p_{r}rs.copy()')
                data_log[num_v] = np.log(data_log[num_v])
                #Calculating the trimmed means
                mean_num_v = stats.trim_mean(data_log[num_v], 0.125, axis=0)
                for dim in cat_dims:
                    data_means = data_log[[dim, num_v]]
                    trimmed_means = data_means.groupby(dim).apply(stats.trim_mean, 0.125)
                    trimmed_means = trimmed_means.reset_index()
                    trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim})
                    trimmed_means["trimmed_mean_"+dim] = trimmed_means["trimmed_mean_"+dim].astype(float)
                    data_log = pd.merge(data_log, trimmed_means, on = [dim], how = 'left')
                for dim1 in cat_dims:
                    for dim2 in cat_dims:
                        if dim1 != dim2: 
                            data_means = data_log[[dim1, dim2, num_v]]
                            trimmed_means = data_means.groupby([dim1, dim2]).apply(stats.trim_mean, 0.125)
                            trimmed_means = trimmed_means.reset_index()
                            trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim1+"_"+dim2})
                            trimmed_means["trimmed_mean_"+dim1+"_"+dim2] = trimmed_means["trimmed_mean_"+dim1+"_"+dim2].astype(float)
                            data_log = pd.merge(data_log, trimmed_means, on = [dim1, dim2], how = 'left')
                #Deleting duplicate combinations (it does not matter whether you combine A to B or B to A)
                data_log = data_log.iloc[:, :-2]
                data_log = data_log.drop(data_log.iloc[:,[-2]], axis='columns')
                #Calculating the expected value (based on a truncated version of the formula)
                data_log["expected_value"] = np.exp((
                                                    #trimmed mean AB
                                                    data_log[data_log.columns[-1]] 
                                                    #trimmed mean AC
                                                    + data_log[data_log.columns[-2]]
                                                    #trimmed mean AB
                                                    +data_log[data_log.columns[-3]] 
                                                    #trimmed mean C 
                                                    - data_log[data_log.columns[-4]]
                                                    #trimmed mean B 
                                                    -data_log[data_log.columns[-5]] 
                                                    #trimmed mean A
                                                    - data_log[data_log.columns[-6]]
                                                    #trimmed mean of all cells
                                                    + mean_num_v))

                data_log["Sales_value"] = np.exp(data_log[num_v])
                #Auxiliary variable for calculating the variance
                data_log["X1"] = pow((data_log["Sales_value"]-data_log["expected_value"]),2)*np.log(data_log["expected_value"])    
                x0 = [1]
                #Calculation of the variance
                def fun(x):
                    return (data_log["X1"]/pow(data_log["expected_value"],x)).sum()-np.log(data_log["expected_value"]).sum()
                res = scipy.optimize.fsolve(fun, 1)
                variance = pow(data_log["expected_value"].mean(), res)
                standard_deviation = pow(variance, 0.5)

                #Calculation of the SelfExp value
                data_log["SelfExp"]=abs((data_log["expected_value"]-data_log["Sales_value"]))/standard_deviation
                data_log["S_Outlier"] = np.where(data_log['SelfExp']>= 2.5, True, False)

                #Performance Evaluation
                Total_Run_Time = time() - start
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basis_Dataset = n
                Length_of_Dataset = len(data_log)
                Std_of_Noise = s
                True_percentage_of_Outliers = p
                TP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==1)])
                FP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==0)])
                TN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==0)])
                FN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==1)])
                
                Sensitivity = TP / (TP + FN)
                Specificity = TN / (TN + FP)
                Accuracy = (TP + TN) / (TP + FP + TN + FN)
                AUC_score = roc_auc_score(data_log["Outlier"], data_log["SelfExp"])

                ds.append((Total_Run_Time, Dataset, Basis_Dataset, Length_of_Dataset, Std_of_Noise, 
                           True_percentage_of_Outliers, TP, FP, TN, FN, Sensitivity, Specificity, Accuracy, AUC_score))

                cols=["Total_Run_Time", "Dataset", "Basis_Dataset", 'Length_of_Dataset', "Std_of_Noise", 
                      "True_percentage_of_Outliers", "TP", "FP", "TN", "FN", "Sensitivity", "Specificity", "Accuracy",
                     "AUC_score"]

                result = pd.DataFrame(ds, columns=cols)
result["Total_Run_Time_shift"] = result.Total_Run_Time.shift(1) 
result["Total_Run_Time_shift"][0] = 0
result["Run_Time"] = result["Total_Run_Time"] - result["Total_Run_Time_shift"]
result = result.drop(["Total_Run_Time", "Total_Run_Time_shift"], axis=1)

evaluation = result[["Basis_Dataset", "Sensitivity", "Specificity", "Accuracy", "AUC_score"]]
evaluation = evaluation.groupby("Basis_Dataset").mean()
evaluation = evaluation.reset_index()
#evaluation.to_excel(r"C:\\__SelfExp_evaluation.xlsx")
#result.to_excel(r"C:\\__SelfExp.xlsx")

# RODD - S60

In [ ]:
start = time()
ds = []
for n in range (2,10):
    for p in [25,100,500]: 
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                num_v = "Sales"
                cat_dims = ["Time", "City", "Product"]
                exec(f'data_log = d{n}_{p}p_{r}rs.copy()')
                data_log[num_v] = np.log(data_log[num_v])
                #Calculating the trimmed means
                mean_num_v = stats.trim_mean(data_log[num_v], 0.2, axis=0)
                for dim in cat_dims:
                    data_means = data_log[[dim, num_v]]
                    trimmed_means = data_means.groupby(dim).apply(stats.trim_mean, 0.2)
                    trimmed_means = trimmed_means.reset_index()
                    trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim})
                    trimmed_means["trimmed_mean_"+dim] = trimmed_means["trimmed_mean_"+dim].astype(float)
                    data_log = pd.merge(data_log, trimmed_means, on = [dim], how = 'left')
                for dim1 in cat_dims:
                    for dim2 in cat_dims:
                        if dim1 != dim2: 
                            data_means = data_log[[dim1, dim2, num_v]]
                            trimmed_means = data_means.groupby([dim1, dim2]).apply(stats.trim_mean, 0.2)
                            trimmed_means = trimmed_means.reset_index()
                            trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim1+"_"+dim2})
                            trimmed_means["trimmed_mean_"+dim1+"_"+dim2] = trimmed_means["trimmed_mean_"+dim1+"_"+dim2].astype(float)
                            data_log = pd.merge(data_log, trimmed_means, on = [dim1, dim2], how = 'left')
                #Deleting duplicate combinations (it does not matter whether you combine A to B or B to A)
                data_log = data_log.iloc[:, :-2]
                data_log = data_log.drop(data_log.iloc[:,[-2]], axis='columns')
                #Calculating the expected value (based on a truncated version of the formula)
                data_log["expected_value"] = np.exp((
                                                    #trimmed mean AB
                                                    data_log[data_log.columns[-1]] 
                                                    #trimmed mean AC
                                                    + data_log[data_log.columns[-2]]
                                                    #trimmed mean AB
                                                    +data_log[data_log.columns[-3]] 
                                                    #trimmed mean C 
                                                    - data_log[data_log.columns[-4]]
                                                    #trimmed mean B 
                                                    -data_log[data_log.columns[-5]] 
                                                    #trimmed mean A
                                                    - data_log[data_log.columns[-6]]
                                                    #trimmed mean of all cells
                                                    + mean_num_v))

                data_log["Sales_value"] = np.exp(data_log[num_v])
                #Auxiliary variable for calculating the variance
                data_log["X1"] = pow((data_log["Sales_value"]-data_log["expected_value"]),2)*np.log(data_log["expected_value"])    
                x0 = [1]
                #Calculation of the variance
                def fun(x):
                    return (data_log["X1"]/pow(data_log["expected_value"],x)).sum()-np.log(data_log["expected_value"]).sum()
                res = scipy.optimize.fsolve(fun, 1)
                variance = pow(data_log["expected_value"].mean(), res)
                standard_deviation = pow(variance, 0.5)

                #Calculation of the SelfExp value
                data_log["SelfExp"]=abs((data_log["expected_value"]-data_log["Sales_value"]))/standard_deviation
                data_log["S_Outlier"] = np.where(data_log['SelfExp']>= 2.5, True, False)

                #Performance Evaluation
                Total_Run_Time = time() - start
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basis_Dataset = n
                Length_of_Dataset = len(data_log)
                Std_of_Noise = s
                True_percentage_of_Outliers = p
                TP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==1)])
                FP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==0)])
                TN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==0)])
                FN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==1)])
                
                Sensitivity = TP / (TP + FN)
                Specificity = TN / (TN + FP)
                Accuracy = (TP + TN) / (TP + FP + TN + FN)
                AUC_score = roc_auc_score(data_log["Outlier"], data_log["SelfExp"])

                ds.append((Total_Run_Time, Dataset, Basis_Dataset, Length_of_Dataset, Std_of_Noise, 
                           True_percentage_of_Outliers, TP, FP, TN, FN, Sensitivity, Specificity, Accuracy, AUC_score))

                cols=["Total_Run_Time", "Dataset", "Basis_Dataset", 'Length_of_Dataset', "Std_of_Noise", 
                      "True_percentage_of_Outliers", "TP", "FP", "TN", "FN", "Sensitivity", "Specificity", "Accuracy",
                     "AUC_score"]

                result = pd.DataFrame(ds, columns=cols)
result["Total_Run_Time_shift"] = result.Total_Run_Time.shift(1) 
result["Total_Run_Time_shift"][0] = 0
result["Run_Time"] = result["Total_Run_Time"] - result["Total_Run_Time_shift"]
result = result.drop(["Total_Run_Time", "Total_Run_Time_shift"], axis=1)

evaluation = result[["Basis_Dataset", "Sensitivity", "Specificity", "Accuracy", "AUC_score"]]
evaluation = evaluation.groupby("Basis_Dataset").mean()
evaluation = evaluation.reset_index()
#evaluation.to_excel(r"C:\\__SelfExp_60_evaluation.xlsx")
#result.to_excel(r"C:\\__SelfExp_60.xlsx")

# RODD - S90

In [ ]:
start = time()
ds = []
for n in range (2,10):
    for p in [25,100,500]: 
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                num_v = "Sales"
                cat_dims = ["Time", "City", "Product"]
                exec(f'data_log = d{n}_{p}p_{r}rs.copy()')
                data_log[num_v] = np.log(data_log[num_v])
                #Calculating the trimmed means
                mean_num_v = stats.trim_mean(data_log[num_v], 0.05, axis=0)
                for dim in cat_dims:
                    data_means = data_log[[dim, num_v]]
                    trimmed_means = data_means.groupby(dim).apply(stats.trim_mean, 0.05)
                    trimmed_means = trimmed_means.reset_index()
                    trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim})
                    trimmed_means["trimmed_mean_"+dim] = trimmed_means["trimmed_mean_"+dim].astype(float)
                    data_log = pd.merge(data_log, trimmed_means, on = [dim], how = 'left')
                for dim1 in cat_dims:
                    for dim2 in cat_dims:
                        if dim1 != dim2: 
                            data_means = data_log[[dim1, dim2, num_v]]
                            trimmed_means = data_means.groupby([dim1, dim2]).apply(stats.trim_mean, 0.05)
                            trimmed_means = trimmed_means.reset_index()
                            trimmed_means = trimmed_means.rename(columns={0:"trimmed_mean_"+dim1+"_"+dim2})
                            trimmed_means["trimmed_mean_"+dim1+"_"+dim2] = trimmed_means["trimmed_mean_"+dim1+"_"+dim2].astype(float)
                            data_log = pd.merge(data_log, trimmed_means, on = [dim1, dim2], how = 'left')
                #Deleting duplicate combinations (it does not matter whether you combine A to B or B to A)
                data_log = data_log.iloc[:, :-2]
                data_log = data_log.drop(data_log.iloc[:,[-2]], axis='columns')
                #Calculating the expected value (based on a truncated version of the formula)
                data_log["expected_value"] = np.exp((
                                                    #trimmed mean AB
                                                    data_log[data_log.columns[-1]] 
                                                    #trimmed mean AC
                                                    + data_log[data_log.columns[-2]]
                                                    #trimmed mean AB
                                                    +data_log[data_log.columns[-3]] 
                                                    #trimmed mean C 
                                                    - data_log[data_log.columns[-4]]
                                                    #trimmed mean B 
                                                    -data_log[data_log.columns[-5]] 
                                                    #trimmed mean A
                                                    - data_log[data_log.columns[-6]]
                                                    #trimmed mean of all cells
                                                    + mean_num_v))

                data_log["Sales_value"] = np.exp(data_log[num_v])
                #Auxiliary variable for calculating the variance
                data_log["X1"] = pow((data_log["Sales_value"]-data_log["expected_value"]),2)*np.log(data_log["expected_value"])    
                x0 = [1]
                #Calculation of the variance
                def fun(x):
                    return (data_log["X1"]/pow(data_log["expected_value"],x)).sum()-np.log(data_log["expected_value"]).sum()
                res = scipy.optimize.fsolve(fun, 1)
                variance = pow(data_log["expected_value"].mean(), res)
                standard_deviation = pow(variance, 0.5)

                #Calculation of the SelfExp value
                data_log["SelfExp"]=abs((data_log["expected_value"]-data_log["Sales_value"]))/standard_deviation
                data_log["S_Outlier"] = np.where(data_log['SelfExp']>= 2.5, True, False)

                #Performance Evaluation
                Total_Run_Time = time() - start
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basis_Dataset = n
                Length_of_Dataset = len(data_log)
                Std_of_Noise = s
                True_percentage_of_Outliers = p
                TP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==1)])
                FP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==0)])
                TN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==0)])
                FN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==1)])
                
                Sensitivity = TP / (TP + FN)
                Specificity = TN / (TN + FP)
                Accuracy = (TP + TN) / (TP + FP + TN + FN)
                AUC_score = roc_auc_score(data_log["Outlier"], data_log["SelfExp"])

                ds.append((Total_Run_Time, Dataset, Basis_Dataset, Length_of_Dataset, Std_of_Noise, 
                           True_percentage_of_Outliers, TP, FP, TN, FN, Sensitivity, Specificity, Accuracy, AUC_score))

                cols=["Total_Run_Time", "Dataset", "Basis_Dataset", 'Length_of_Dataset', "Std_of_Noise", 
                      "True_percentage_of_Outliers", "TP", "FP", "TN", "FN", "Sensitivity", "Specificity", "Accuracy",
                     "AUC_score"]

                result = pd.DataFrame(ds, columns=cols)
result["Total_Run_Time_shift"] = result.Total_Run_Time.shift(1) 
result["Total_Run_Time_shift"][0] = 0
result["Run_Time"] = result["Total_Run_Time"] - result["Total_Run_Time_shift"]
result = result.drop(["Total_Run_Time", "Total_Run_Time_shift"], axis=1)

evaluation = result[["Basis_Dataset", "Sensitivity", "Specificity", "Accuracy", "AUC_score"]]
evaluation = evaluation.groupby("Basis_Dataset").mean()
evaluation = evaluation.reset_index()
#evaluation.to_excel(r"C:\\__SelfExp_90_evaluation.xlsx")
#result.to_excel(r"C:\\__SelfExp_90.xlsx")

# RODD - Median

In [6]:
start = time()
ds = []
for n in range (2,10):
    for p in [25,100,500]: 
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                num_v = "Sales"
                cat_dims = ["Time", "City", "Product"]
                exec(f'data_log = d{n}_{p}p_{r}rs.copy()')
                data_log[num_v] = scipy.stats.rankdata(data_log[num_v])
                data_log[num_v] = np.log(data_log[num_v])
                #Calculating the trimmed means
                mean_num_v = np.median(data_log[num_v])
                for dim in cat_dims:
                    data_medians = data_log[[dim, num_v]]
                    medians = data_medians.groupby(dim).apply(np.median)
                    medians = medians.reset_index()
                    medians = medians.rename(columns={0:"median_"+dim})
                    medians["median_"+dim] = medians["median_"+dim].astype(float)
                    data_log = pd.merge(data_log, medians, on = [dim], how = 'left')
                for dim1 in cat_dims:
                    for dim2 in cat_dims:
                        if dim1 != dim2: 
                            data_medians = data_log[[dim1, dim2, num_v]]
                            medians = data_medians.groupby([dim1, dim2]).apply(np.median)
                            medians = medians.reset_index()
                            medians = medians.rename(columns={0:"median_"+dim1+"_"+dim2})
                            medians["median_"+dim1+"_"+dim2] = medians["median_"+dim1+"_"+dim2].astype(float)
                            data_log = pd.merge(data_log, medians, on = [dim1, dim2], how = 'left')
                #Deleting duplicate combinations (it does not matter whether you combine A to B or B to A)
                data_log = data_log.iloc[:, :-2]
                data_log = data_log.drop(data_log.iloc[:,[-2]], axis='columns')
                #Calculating the expected value (based on a truncated version of the formula)
                data_log["expected_value"] = np.exp((
                                                    #trimmed mean AB
                                                    data_log[data_log.columns[-1]] 
                                                    #trimmed mean AC
                                                    + data_log[data_log.columns[-2]]
                                                    #trimmed mean AB
                                                    +data_log[data_log.columns[-3]] 
                                                    #trimmed mean C 
                                                    - data_log[data_log.columns[-4]]
                                                    #trimmed mean B 
                                                    -data_log[data_log.columns[-5]] 
                                                    #trimmed mean A
                                                    - data_log[data_log.columns[-6]]
                                                    #trimmed mean of all cells
                                                    + mean_num_v))

                data_log["Sales_value"] = np.exp(data_log[num_v])
                #Auxiliary variable for calculating the variance
                data_log["X1"] = pow((data_log["Sales_value"]-data_log["expected_value"]),2)*np.log(data_log["expected_value"])    
                x0 = [1]
                #Calculation of the variance
                def fun(x):
                    return (data_log["X1"]/pow(data_log["expected_value"],x)).sum()-np.log(data_log["expected_value"]).sum()
                res = scipy.optimize.fsolve(fun, 1)
                variance = pow(data_log["expected_value"].mean(), res)
                standard_deviation = pow(variance, 0.5)

                #Calculation of the SelfExp value
                data_log["SelfExp"]=abs((data_log["expected_value"]-data_log["Sales_value"]))/standard_deviation
                data_log["S_Outlier"] = np.where(data_log['SelfExp']>= 2.5, True, False)

                #Performance Evaluation
                Total_Run_Time = time() - start
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basis_Dataset = n
                Length_of_Dataset = len(data_log)
                Std_of_Noise = s
                True_percentage_of_Outliers = p
                TP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==1)])
                FP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==0)])
                TN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==0)])
                FN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==1)])
                
                Sensitivity = TP / (TP + FN)
                Specificity = TN / (TN + FP)
                Accuracy = (TP + TN) / (TP + FP + TN + FN)
                AUC_score = roc_auc_score(data_log["Outlier"], data_log["SelfExp"])
                
                ds.append((Total_Run_Time, Dataset, Basis_Dataset, Length_of_Dataset, Std_of_Noise, 
                           True_percentage_of_Outliers, TP, FP, TN, FN, Sensitivity, Specificity, Accuracy, AUC_score))

                cols=["Total_Run_Time", "Dataset", "Basis_Dataset", 'Length_of_Dataset', "Std_of_Noise", 
                      "True_percentage_of_Outliers", "TP", "FP", "TN", "FN", "Sensitivity", "Specificity", "Accuracy",
                     "AUC_score"]

                result = pd.DataFrame(ds, columns=cols)
result["Total_Run_Time_shift"] = result.Total_Run_Time.shift(1) 
result["Total_Run_Time_shift"][0] = 0
result["Run_Time"] = result["Total_Run_Time"] - result["Total_Run_Time_shift"]
result = result.drop(["Total_Run_Time", "Total_Run_Time_shift"], axis=1)

evaluation = result[["Basis_Dataset", "Sensitivity", "Specificity", "Accuracy", "AUC_score"]]
evaluation = evaluation.groupby("Basis_Dataset").mean()
evaluation = evaluation.reset_index()
#evaluation.to_excel(r"C:\\__Median_evaluation.xlsx")
#result.to_excel(r"C:\\__Median.xlsx")

# RODD - RF

In [3]:
start = time()
ds = []
for n in range (2,10):
    for p in [25,100,500]: 
        for r in range (1,31):
            for s in [10, 15, 20, 25, 30]:
                exec(f'data_log = d{n}_{p}p_{r}rs.copy()')
                num_v = "Sales"

                #Training a RF
                features = data_log[["Time", "City", "Product", "Outlier"]]
                train_features, test_features, train_labels, test_labels = train_test_split(features, data_log[num_v], test_size = 0.1, random_state = 42)
                train_features_en = train_features.drop("Outlier", axis = 1)
                train_features_en = pd.get_dummies(train_features_en)
                test_features_en = test_features.drop("Outlier", axis = 1)
                test_features_en = pd.get_dummies(test_features_en)
                rf = RandomForestRegressor(n_estimators = 20, random_state = 42)
                rf.fit(train_features_en, train_labels)                
                predictions_train = rf.predict(train_features_en)
                predictions_test = rf.predict(test_features_en)
                train_features["expected_value"] = predictions_train
                test_features["expected_value"] = predictions_test
                
                predictions = train_features.append(test_features)
                data_log = pd.merge(data_log, predictions, on = ["Time", "City", "Product", "Outlier"], how = 'left') 

                #Auxiliary variable for calculating the variance
                data_log["X1"] = pow((data_log[num_v]-data_log["expected_value"]),2)*np.log(data_log["expected_value"])    
                x0 = [1]
                #Calculation of the variance
                def fun(x):
                    return (data_log["X1"]/pow(data_log["expected_value"],x)).sum()-np.log(data_log["expected_value"]).sum()
                res = scipy.optimize.fsolve(fun, 1)
                variance = pow(data_log["expected_value"].mean(), res)
                standard_deviation = pow(variance, 0.5)

                #Calculation of the SelfExp value
                data_log["SelfExp"]=abs((data_log["expected_value"]-data_log[num_v]))/standard_deviation
                data_log["S_Outlier"] = np.where(data_log['SelfExp']>= 2.5, True, False)

                #Performance Evaluation
                Total_Run_Time = time() - start
                exec(f'Dataset = "d{n}_{p}p_{r}rs"')
                Basis_Dataset = n
                Length_of_Dataset = len(data_log)
                Std_of_Noise = s
                True_percentage_of_Outliers = p
                TP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==1)])
                FP = len(data_log[(data_log["S_Outlier"]==True) & (data_log["Outlier"]==0)])
                TN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==0)])
                FN = len(data_log[(data_log["S_Outlier"]==False) & (data_log["Outlier"]==1)])
                
                Sensitivity = TP / (TP + FN)
                Specificity = TN / (TN + FP)
                Accuracy = (TP + TN) / (TP + FP + TN + FN)
                AUC_score = roc_auc_score(data_log["Outlier"], data_log["SelfExp"])

                ds.append((Total_Run_Time, Dataset, Basis_Dataset, Length_of_Dataset, Std_of_Noise, 
                           True_percentage_of_Outliers, TP, FP, TN, FN, Sensitivity, Specificity, Accuracy, AUC_score))

                cols=["Total_Run_Time", "Dataset", "Basis_Dataset", 'Length_of_Dataset', "Std_of_Noise", 
                      "True_percentage_of_Outliers", "TP", "FP", "TN", "FN", "Sensitivity", "Specificity", "Accuracy",
                     "AUC_score"]

                result = pd.DataFrame(ds, columns=cols)
result["Total_Run_Time_shift"] = result.Total_Run_Time.shift(1) 
result["Total_Run_Time_shift"][0] = 0
result["Run_Time"] = result["Total_Run_Time"] - result["Total_Run_Time_shift"]
result = result.drop(["Total_Run_Time", "Total_Run_Time_shift"], axis=1)

evaluation = result[["Basis_Dataset", "Sensitivity", "Specificity", "Accuracy", "AUC_score"]]
evaluation = evaluation.groupby("Basis_Dataset").mean()
evaluation = evaluation.reset_index()
#evaluation.to_excel(r"C:\\__RF_evaluation.xlsx")
#result.to_excel(r"C:\\__RF.xlsx")